# Imports

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import cv2 as cv
import typing as t
import pandas as pd
import pathlib as pb
import numpy as np
import sklearn as sk
import random as rnd
import matplotlib.pyplot as plt
import torch
import torchvision as tv
import torch.utils.data as data
from torch import Tensor
import lightning.pytorch as pl

In [ ]:
import domino.data
import domino.game
import domino.model
import domino.vision
import domino.preprocess
from domino.model import CellClassifier

# Environment Setup

In [ ]:
# Predefined expected paths
ROOT_PATH: pb.Path = pb.Path('.')
DATA_PATH: pb.Path = ROOT_PATH / '..' / 'data'
HELP_PATH: pb.Path = ROOT_PATH / 'template'
CACHE_DIR: pb.Path = ROOT_PATH / '.cache'
MODEL_DIR: pb.Path = ROOT_PATH / '.model'
SKLRN_DIR: pb.Path = MODEL_DIR / 'sklearn'
TORCH_DIR: pb.Path = MODEL_DIR / 'torch'

# Pretrained model weights
FOREST_MODEL_PATH: pb.Path = SKLRN_DIR / 'balanced-random-forest.joblib'
RESNET_MODEL_PATH: pb.Path = TORCH_DIR / 'resnet50-finetune-wip.ckpt'
SVM_MODEL_PATH: pb.Path    = SKLRN_DIR / 'svm.joblib'

# Main dataset for the given tasks
TEST_PATH: pb.Path       = DATA_PATH / 'test'  / 'input'
TRAIN_PATH: pb.Path      = DATA_PATH / 'train' / 'input'
VALID_PATH: pb.Path      = DATA_PATH / 'valid' / 'input'

# Auxiliary dataset made of grid cell patches
CELL_DATA_PATH: pb.Path  = DATA_PATH      / 'cell'
CELL_TRAIN_PATH: pb.Path = CELL_DATA_PATH / 'train'
CELL_TEST_PATH: pb.Path  = CELL_DATA_PATH / 'test'

# Obtain deterministic results
SEED = 42
rnd.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
generator: torch.Generator = torch.Generator().manual_seed(SEED)

# Configure current environment
torch.hub.set_dir(CACHE_DIR)
torch.set_float32_matmul_precision('medium')

In [ ]:
# Load datasets for the task
train_dataset = domino.data.DDDGameDataset(path_input=TRAIN_PATH,
                                           path_template=HELP_PATH,
                                           path_output=TRAIN_PATH / '..' / 'output',
                                           train=True)
valid_dataset = domino.data.DDDGameDataset(path_input=VALID_PATH,
                                           path_template=HELP_PATH,
                                           path_output=VALID_PATH / '..' / 'output',
                                           train=True)
test_dataset  = domino.data.DDDGameDataset(path_input=TEST_PATH,
                                           path_template=HELP_PATH,
                                           path_output=TEST_PATH / '..' / 'output',
                                           train=False)

# Construct the possible classifiers
classifiers: t.List[CellClassifier] = [
    CellClassifier.load_checkpoint(SVM_MODEL_PATH, 'svm'),
    CellClassifier.load_checkpoint(FOREST_MODEL_PATH, 'forest'),
    CellClassifier.load_checkpoint(RESNET_MODEL_PATH, 'resnet'),
]

# Construct task solver
task_solver = domino.game.DoubleDoubleDominoesTask(
    dataset_path=TRAIN_PATH,
    template_path=HELP_PATH,
    output_path=TRAIN_PATH / '..' / 'output',
    classifiers=classifiers,
    cell_splitter='hough',
    show_matrix=False,
    show_image=False,
    train=True,
    debug=True,
)

In [ ]:
task_solver.solve()